### Edgar Moises Hernandez-Gonzalez
#### 20-04-20-26-08-20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFFT (3D)
##### Entrada 990x31

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
# from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [3]:
x_train = pd.read_csv("../Datos/MI-EEG-A09T.csv", header=None)
x_test = pd.read_csv("../Datos/MI-EEG-A09E.csv", header=None)
y_test = pd.read_csv("../Datos/EtiquetasA09E.csv", header=None)

In [4]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 22000)
(288, 1)


In [5]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 4


In [ ]:
#codificar etiquetas con valores de 0 a nClases-1
# y_test = LabelEncoder().fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# las etiquetas en categorical para poder entrenar data augmentation
# y_test_categorical = to_categorical(y_test)

In [6]:
#unir 22 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],990,31))
  temporal = np.zeros((990,31))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [7]:
#llamar a unir_espectrogramas_vertical(data)
inicio = time.time()

x_train = unir_espectrogramas_vertical(x_train)
x_test = unir_espectrogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
0
100
200
Tiempo: 7.495007753372192


In [8]:
print(x_test.shape)

(288, 990, 31)


In [9]:
x = []
x.append(np.ceil(np.max(x_train)))
# esta linea solo se ocupa si se va a predecir un modelo creado con CV
# x.append(np.ceil(np.max(x_test)))

In [10]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x))

In [11]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 990, 31, 1)


In [12]:
model = load_model('../Modelos/STFT09.hdf5')

In [13]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 990, 31, 16)       160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 495, 15, 16)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 495, 15, 16)       2320      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 247, 7, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 27664)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               3541120   
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [16]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.19689345359802246


In [14]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.5162
accuracy: 0.9306


In [15]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [16]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.8611 


In [17]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[65  7]
 [ 3 69]]


In [17]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,990,31,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.003988742828369141
